Bitcoin Price Analysis with CoinGecko API
==========================================

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo

Installing and Importing CoinGecko API

In [2]:
# Install pycoingecko if needed
!pip install pycoingecko

try:
    from pycoingecko import CoinGeckoAPI
    print("CoinGecko API imported successfully")
except ImportError:
    print("PyCoingecko not installed. Run: pip install pycoingecko")

CoinGecko API imported successfully


Fetching Bitcoin Market Data

In [3]:
try:
    # Initialize CoinGecko API
    cg = CoinGeckoAPI()
    print("CoinGecko API initialized")

    # Get Bitcoin market chart data for past 30 days
    print("Fetching Bitcoin price data for past 30 days...")
    bitcoin_data = cg.get_coin_market_chart_by_id(
        id='bitcoin',
        vs_currency='usd',
        days=30
    )

    print(f"Successfully fetched Bitcoin data")
    print(f"Data keys: {list(bitcoin_data.keys())}")
    print(f"Price data points: {len(bitcoin_data['prices'])}")

except Exception as e:
    print(f"Error fetching data: {e}")

CoinGecko API initialized
Fetching Bitcoin price data for past 30 days...
Successfully fetched Bitcoin data
Data keys: ['prices', 'market_caps', 'total_volumes']
Price data points: 721


Converting Data to DataFrame

In [4]:
try:
    # Convert price data to DataFrame
    data = pd.DataFrame(bitcoin_data['prices'], columns=['TimeStamp', 'Price'])
    print(f"Created DataFrame with shape: {data.shape}")

    # Convert timestamp to datetime
    data['Date'] = pd.to_datetime(data['TimeStamp'], unit='ms')
    print("Converted timestamps to datetime")

    # Display basic info
    print(f"\nData Info:")
    print(f"   Date range: {data['Date'].min()} to {data['Date'].max()}")
    print(f"   Price range: ${data['Price'].min():.2f} to ${data['Price'].max():.2f}")
    print(f"   Average price: ${data['Price'].mean():.2f}")

    # Show sample data
    print(f"\nSample data:")
    print(data[['Date', 'Price']].head())
except Exception as e:
    print(f"Error processing data: {e}")

Created DataFrame with shape: (721, 2)
Converted timestamps to datetime

Data Info:
   Date range: 2025-05-11 04:04:33.743000 to 2025-06-10 03:37:37
   Price range: $100983.60 to $111806.60
   Average price: $105826.54

Sample data:
                     Date          Price
0 2025-05-11 04:04:33.743  104056.371706
1 2025-05-11 05:04:41.501  104155.909998
2 2025-05-11 06:04:51.730  103870.884158
3 2025-05-11 07:04:52.454  103829.809188
4 2025-05-11 08:06:18.310  103593.918385


Preparing Candlestick Data

In [5]:
try:
    # Group by date to create OHLC (Open, High, Low, Close) data
    candlestick_data = data.groupby(data.Date.dt.date).agg({
        'Price': ['min', 'max', 'first', 'last']
    })

    # Flatten column names
    candlestick_data.columns = ['min', 'max', 'first', 'last']
    candlestick_data.reset_index(inplace=True)

    print(f"Candlestick data shape: {candlestick_data.shape}")
    print(f"Sample candlestick data:")
    print(candlestick_data.head())
except Exception as e:
    print(f"Error creating candlestick data: {e}")

Candlestick data shape: (31, 5)
Sample candlestick data:
         Date            min            max          first           last
0  2025-05-11  103593.918385  104642.107893  104056.371706  103785.595390
1  2025-05-12  101758.108960  104632.602361  104083.022801  102853.644071
2  2025-05-13  101749.060160  104778.262185  102822.916795  104235.965957
3  2025-05-14  103236.433831  104184.490393  104184.490393  103500.558030
4  2025-05-15  101775.761206  103977.536746  103546.469182  103553.935064


Creating Interactive Candlestick Chart

In [6]:
try:
    # Create candlestick chart using Plotly
    fig = go.Figure(data=[go.Candlestick(
        x=candlestick_data.index,
        open=candlestick_data['first'],
        high=candlestick_data['max'],
        low=candlestick_data['min'],
        close=candlestick_data['last']
    )])

    # Update layout
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        xaxis_title='Date',
        yaxis_title='Price (USD $)',
        title='Bitcoin Candlestick Chart Over Past 30 Days',
        template='plotly_white'
    )

    print("Candlestick chart created successfully")

    # Save chart as HTML file
    pyo.plot(fig, filename='bitcoin_candlestick_chart.html', auto_open=False)
    print("Chart saved as 'bitcoin_candlestick_chart.html'")

    # Display chart (in Jupyter notebook this would show inline)
    fig.show()
except Exception as e:
    print(f"Error creating chart: {e}")

Candlestick chart created successfully
Chart saved as 'bitcoin_candlestick_chart.html'


Additional Market Analysis

In [7]:
try:
    # Price statistics
    price_change = data['Price'].iloc[-1] - data['Price'].iloc[0]
    price_change_pct = (price_change / data['Price'].iloc[0]) * 100
    volatility = data['Price'].std()

    print(f"Price Analysis:")
    print(f"   Starting price: ${data['Price'].iloc[0]:.2f}")
    print(f"   Ending price: ${data['Price'].iloc[-1]:.2f}")
    print(f"   Price change: ${price_change:.2f} ({price_change_pct:.2f}%)")
    print(f"   Volatility (std): ${volatility:.2f}")

    # Daily price changes
    data['Daily_Change'] = data['Price'].pct_change() * 100
    daily_changes = data['Daily_Change'].dropna()

    print(f"\nDaily Change Analysis:")
    print(f"   Average daily change: {daily_changes.mean():.2f}%")
    print(f"   Max daily gain: {daily_changes.max():.2f}%")
    print(f"   Max daily loss: {daily_changes.min():.2f}%")
except Exception as e:
    print(f"Error in analysis: {e}")

Price Analysis:
   Starting price: $104056.37
   Ending price: $109468.52
   Price change: $5412.14 (5.20%)
   Volatility (std): $2392.26

Daily Change Analysis:
   Average daily change: 0.01%
   Max daily gain: 1.29%
   Max daily loss: -2.38%


Creating Simple Price Trend Chart

In [8]:
try:
    # Create simple line chart
    fig_line = go.Figure()

    fig_line.add_trace(go.Scatter(
        x=data['Date'],
        y=data['Price'],
        mode='lines',
        name='Bitcoin Price',
        line=dict(color='orange', width=2)
    ))

    fig_line.update_layout(
        title='Bitcoin Price Trend - Past 30 Days',
        xaxis_title='Date',
        yaxis_title='Price (USD $)',
        template='plotly_white',
        hovermode='x unified'
    )

    # Save line chart
    pyo.plot(fig_line, filename='bitcoin_price_trend.html', auto_open=False)
    print("Price trend chart saved as 'bitcoin_price_trend.html'")

    fig_line.show()
except Exception as e:
    print(f"Error creating line chart: {e}")

Price trend chart saved as 'bitcoin_price_trend.html'
